In [ ]:
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Optional, Union
from pydantic import BaseModel, Field

class RelationshipAnalysis(BaseModel):
    id_bb_company: str
    name: str
    sector: str
    ticker: str
    ranking: float
    analysis: Dict[str, str]

class EnhancedECTResponse(BaseModel):
    company_analysis: Dict[str, dict]
    customer_analysis: Optional[Dict[str, RelationshipAnalysis]]
    supplier_analysis: Optional[Dict[str, RelationshipAnalysis]]
    metadata: dict

    class Config:
        frozen = True

class ECTAnalyzer:
    def _get_relationship_db(self) -> RelationDB:
        quarter = f"Q{self.latest_doc.quarter_s}"
        year = int(self.latest_doc.year_s)
        return RelationDB(year=year, quarter=quarter, use_cache=True)

    def _generate_customer_prompt(self, ect: str, company_name: str, sector: str) -> str:
        return f"""Analyze the earnings call for {company_name} (Sector: {sector}). 
Focus on:
1. Future revenue prospects and growth trajectory
2. Key growth drivers and market opportunities
3. Customer demand trends and market share
4. Strategic initiatives and expansion plans

Earnings call text:
{ect}"""

    def _generate_supplier_prompt(self, ect: str, company_name: str, sector: str) -> str:
        return f"""Analyze the earnings call for {company_name} (Sector: {sector}). 
Focus on:
1. Pricing power and cost management
2. Supply chain health and inventory levels
3. Production capacity and utilization
4. Capital expenditure plans
5. Product development pipeline

Earnings call text:
{ect}"""

    async def get_enhanced_ect(
        self,
        company_id: str,
        include_relationships: bool = True
    ) -> EnhancedECTResponse:
        # Get main company ECT
        documents = await self.get_earnings_by_company_id(company_id)
        if not documents:
            raise ValueError("No documents found")

        self.latest_doc = max(documents, key=lambda x: datetime.strptime(x.document_date_s, "%Y-%m-%d"))
        content = await self.get_earnings_content(self.latest_doc)
        
        # Main company analysis
        prompts = self.generate_ect_prompts(content, company_id)
        company_responses = await self.execute_prompts(prompts)
        
        company_analysis = {}
        for response in company_responses:
            if response["confidence"] > self.config.llm.confidence_threshold:
                company_analysis.update(response["result"])

        if not include_relationships:
            return EnhancedECTResponse(
                company_analysis=company_analysis,
                customer_analysis=None,
                supplier_analysis=None,
                metadata=self._create_metadata(self.latest_doc)
            )

        # Get relationship data
        rel_db = self._get_relationship_db()
        company_ticker = self.latest_doc.tickers_s[0]
        
        customers = rel_db.get_customers_by_ticker(company_ticker)
        suppliers = rel_db.get_suppliers_by_ticker(company_ticker)

        customer_analysis = await self._analyze_relationships(
            customers, "customer"
        )
        supplier_analysis = await self._analyze_relationships(
            suppliers, "supplier"
        )

        return EnhancedECTResponse(
            company_analysis=company_analysis,
            customer_analysis=customer_analysis,
            supplier_analysis=supplier_analysis,
            metadata=self._create_metadata(self.latest_doc)
        )

    async def _analyze_relationships(
        self,
        relationships: Dict[str, CompanyInfo],
        relationship_type: str
    ) -> Dict[str, RelationshipAnalysis]:
        analysis = {}
        
        for bbg_cid, info in relationships.items():
            docs = await self.get_earnings_by_company_id(bbg_cid)
            if not docs:
                continue
                
            latest = max(docs, key=lambda x: datetime.strptime(x.document_date_s, "%Y-%m-%d"))
            content = await self.get_earnings_content(latest)
            
            prompt_func = (
                self._generate_customer_prompt 
                if relationship_type == "customer" 
                else self._generate_supplier_prompt
            )
            
            prompt = prompt_func(content, info.name, info.sector)
            response = await self.execute_prompts([{
                "prompt": prompt,
                "company": bbg_cid,
                "type": relationship_type,
                "model": self.config.llm.llm_name
            }])
            
            if response and response[0]["confidence"] > self.config.llm.confidence_threshold:
                analysis[bbg_cid] = RelationshipAnalysis(
                    id_bb_company=bbg_cid,
                    name=info.name,
                    sector=info.sector,
                    ticker=info.ticker,
                    ranking=info.ranking,
                    analysis=response[0]["result"]
                )

        return analysis